#### Connect with database and ask some basic questions

In [1]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent,SQLDatabaseToolkit
import os

c:\Users\a\Desktop\MyProjects\LLMProjects\chat_db_sql\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['OPENAI_API_KEY']=
from langchain_openai import ChatOpenAI
llm=ChatOpenAI()

In [3]:
db_user = "root"
db_password = "root"
db_host = "localhost"
db_name = "store_tshirts"

db = SQLDatabase.from_uri(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [4]:
agent_executor = create_sql_agent(llm, toolkit = SQLDatabaseToolkit(llm=llm,db=db),verbose=True)

#### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [5]:
examples = [
    {'input' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'query' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'"},
    {'input': "How much is the total price of the inventory for all S-size t-shirts?",
     'query':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'"},
    {'input': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'query' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """} ,
     {'input' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'query': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'"},
    {'input': "How many white color Levi's shirt I have?",
     'query' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'" }
]

### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in Chroma DB
- Retrieve the the top most Semantically close example from the vector store

In [6]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [7]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=2,
    input_keys=["input"],
)

In [47]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
If the word 'discount' is given in the input query the discounts table also using the t_shirt_id from the t_shirts table and subtract the price.
In "t_shirts" table "price" column refers to price for a single t shirt and "stock_quantity" refers to number of t-shirts.
In "discounts" table "pct_discount" refers to the percent of discount offered.
If there is no corresponding entry in "discounts" table give the "price" column value in  "t_shirts" table.You can return "There is no discount" as the answer .

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [48]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [49]:
toolkit=SQLDatabaseToolkit(llm=llm,db=db)
agent = create_sql_agent(
    llm=llm,
    #db=db,
    toolkit = toolkit,
    prompt=full_prompt,
    verbose=True,
    agent_type='openai-tools',
)


In [50]:
agent.invoke({"input": "What is the price for one Levi M size black t-shirt after discount?"},verbose=True)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `SELECT price FROM t_shirts WHERE brand = 'Levi' AND size = 'M' AND color = 'black'`


SELECT price FROM t_shirts WHERE brand = 'Levi' AND size = 'M' AND color = 'black';
Invoking: `sql_db_query` with `SELECT price, (CASE WHEN d.pct_discount IS NULL THEN price ELSE price - (price * d.pct_discount / 100) END) AS discounted_price FROM t_shirts LEFT JOIN discounts d ON t_shirts.t_shirt_id = d.t_shirt_id WHERE brand = 'Levi' AND size = 'M' AND color = 'black'`


[(48, Decimal('48'))]The price for one Levi M size black t-shirt after discount is $48.

> Finished chain.


{'input': 'What is the price for one Levi M size black t-shirt after discount?',
 'output': 'The price for one Levi M size black t-shirt after discount is $48.'}

In [51]:
agent.invoke({"input": "What is the price for one Van Huesen L size white t-shirt after discount?"},verbose=True)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `SELECT t.price - (t.price * d.pct_discount / 100) AS discounted_price FROM t_shirts t LEFT JOIN discounts d ON t.t_shirt_id = d.t_shirt_id WHERE t.brand = 'Van Huesen' AND t.size = 'L' AND t.color = 'White' LIMIT 1`


[(Decimal('21.250000'),)]The price for one Van Heusen L size white t-shirt after discount is $21.25.

> Finished chain.


{'input': 'What is the price for one Van Huesen L size white t-shirt after discount?',
 'output': 'The price for one Van Heusen L size white t-shirt after discount is $21.25.'}